In [2]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

# for reading and displaying images
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
# %matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# torchvision for pre-trained models
from torchvision import models
import argparse
from collections import Counter
import time
import cv2
import os

import numpy as np

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

import cv2
import pandas as pd

In [ ]:
print(os.listdir('/content/gdrive/MyDrive/mini-MIAS/images_processed'))

['fatty', 'fatty_glandular', 'dense_glandular']


In [ ]:
# Make the validation directories
try:
    os.makedirs("/content/gdrive/MyDrive/mini-MIAS/val/fatty")
    os.makedirs("/content/gdrive/MyDrive/mini-MIAS/val/fatty_glandular")
    os.makedirs("/content/gdrive/MyDrive/mini-MIAS/val/dense_glandular")
except OSError:
    print ("Creation of the directory %s failed")
else:
    print ("Successfully created the directory %s ")

# Create validation folder
fatty_train='/content/gdrive/MyDrive/mini-MIAS/images_processed/dense_glandular'
cat_train = base_dir + "train/Cat/"
cat_val = base_dir + "val/Cat/"
dog_train = base_dir + "train/Dog/"
dog_val = base_dir + "val/Dog/"

# cat_files = os.listdir(cat_train)
# dog_files = os.listdir(dog_train)

# # This will put 1000 images from the two training folders
# # into their respective validation folders

# for f in cat_files:
#     validationCatsSearchObj = re.search("5\d\d\d", f)
#     if validationCatsSearchObj:
#         shutil.move(f'{cat_train}/{f}', cat_val)

# for f in dog_files:
#     validationCatsSearchObj = re.search("5\d\d\d", f)
#     if validationCatsSearchObj:
#         shutil.move(f'{dog_train}/{f}', dog_val)

Successfully created the directory %s 


In [4]:
from torchvision import transforms, datasets, models

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.95, 1.0)),
        #transforms.RandomRotation(degrees=15),
        # transforms.ColorJitter(),
        #transforms.RandomHorizontalFlip(),
        #transforms.CenterCrop(size=224),  # Image net standards
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
        
      
    ]),

    'val': transforms.Compose([
       transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])

    ]),

    'test': transforms.Compose([
       transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])

    ]),
  

    }

In [ ]:
# from torch.utils.data import DataLoader, sampler, random_split

# batch_size = 16

# all_data = datasets.ImageFolder('/content/gdrive/MyDrive/mini-MIAS/images_processed',transform=image_transforms['train'])
# # print(len(all_data))
# # train_data_len = int(len(all_data)*0.8)
# # valid_data_len = int((len(all_data) - train_data_len)/2)
# # test_data_len = int(len(all_data) - train_data_len - valid_data_len)
# # train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
# # train_data.dataset.transform = image_transforms['train']
# # val_data.dataset.transform = image_transforms['val']
# # test_data.dataset.transform = image_transforms['test']
# # print(len(train_data), len(val_data), len(test_data))

# train_loader = DataLoader(all_data, batch_size=batch_size, shuffle=True)
# # val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
# # test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

322


In [6]:
from torch.utils.data import DataLoader, sampler, random_split

batch_size = 8

all_data = datasets.ImageFolder('/content/gdrive/MyDrive/mini-MIAS/images_processed')
print(len(all_data))
train_data_len = int(len(all_data)*0.8)
valid_data_len = int((len(all_data) - train_data_len)/2)
test_data_len = int(len(all_data) - train_data_len - valid_data_len)
train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
train_data.dataset.transform = image_transforms['train']
val_data.dataset.transform = image_transforms['train']
test_data.dataset.transform = image_transforms['train']
print(len(train_data), len(val_data), len(test_data))

train_loader = DataLoader(train_data, batch_size=2, shuffle=True)

val_loader = DataLoader(val_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)
print(len(train_loader),len(test_loader),len(val_loader))

322
257 32 33
129 3 4


In [ ]:
trainiter = iter(test_loader)
features_train, labels_train = next(trainiter)
print(features_train.shape, labels_train.shape)

torch.Size([8, 3, 299, 299]) torch.Size([8])


In [8]:
model = models.resnet50(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False
num_ftrs =model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3)

# num_ftrs


In [9]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)


In [ ]:
model

In [10]:
for name, child in model.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [13]:
# model = model.to(device)
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler


from torch.optim import Adam

optimizer=optim.SGD(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_func=nn.CrossEntropyLoss()

# loss_fn = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [14]:
train_count=322
test_count=33

In [ ]:
num_epochs=20

best_accuracy=0.0
pred=[]
true=[]
for epoch in range(num_epochs):

  model.train()
  train_accuracy=0.0
  train_loss=0.0
  

  for i, (images,labels) in enumerate(train_loader):


    optimizer.zero_grad()
    output=model(images)
    loss=loss_func(output,labels)
    loss.backward()
    optimizer.step()

    train_loss+=loss.item()
    
    _, prediction=torch.max(output.data,1)

        
   
    train_accuracy+=(prediction==labels).sum().item()

    prediction=prediction.cpu().numpy()
    labels=labels .cpu().numpy()
    prediction = np.reshape(prediction,(len(prediction),1))
    labels = np.reshape(labels,(len(prediction),1))

    for i in range(len(prediction)):
      pred.append(prediction[i])
      true.append(labels[i])
            
        
  train_accuracy=train_accuracy/train_count
  train_loss=train_loss/train_count

  model.eval()
    
  test_accuracy=0.0
  for i, (images,labels) in enumerate(test_loader):
      if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())
            
      outputs=model(images)
      _, prediction=torch.max(outputs.data,1)
      test_accuracy+=(prediction==labels).sum().item()
 
    
  test_accuracy=test_accuracy/test_count
    
    
  print('Epoch: ',epoch,' Train Loss: ',train_loss,' Train Accuracy: ',train_accuracy,'Test Accuracy: ',test_accuracy)

  if test_accuracy>best_accuracy: 
        torch.save(model.state_dict(),'/content/gdrive/MyDrive/mini-MIAS/best_checkpoint50.model')
        best_accuracy=test_accuracy

Epoch:  0  Train Loss:  0.4513636731212924  Train Accuracy:  0.2857142857142857 Test Accuracy:  0.36363636363636365
Epoch:  1  Train Loss:  0.40806738948970106  Train Accuracy:  0.38509316770186336 Test Accuracy:  0.30303030303030304
Epoch:  2  Train Loss:  0.3789783134038404  Train Accuracy:  0.45962732919254656 Test Accuracy:  0.30303030303030304
Epoch:  3  Train Loss:  0.34457781475893456  Train Accuracy:  0.468944099378882 Test Accuracy:  0.6666666666666666
Epoch:  4  Train Loss:  0.32566952376817326  Train Accuracy:  0.4906832298136646 Test Accuracy:  0.5757575757575758
Epoch:  5  Train Loss:  0.34093833747117414  Train Accuracy:  0.4782608695652174 Test Accuracy:  0.7878787878787878
Epoch:  6  Train Loss:  0.2983516359366245  Train Accuracy:  0.5279503105590062 Test Accuracy:  0.5454545454545454
Epoch:  7  Train Loss:  0.30357375713238804  Train Accuracy:  0.5124223602484472 Test Accuracy:  0.7575757575757576
Epoch:  8  Train Loss:  0.29094687034809813  Train Accuracy:  0.5527950